In [1]:
from api_circle import CircleAPI
from api_circle import render_json
from api_intercom import IntercomAPI
from migrate_circle_posts_to_intercom import MigrateCirclePostsToIntercom
import pandas as pd

In [2]:
migration = MigrateCirclePostsToIntercom()

# migrate all collections

In [ ]:
## delete existing collections
# migration.delete_all_intercom_collections(are_you_sure=False)

In [ ]:
#res_collections = migration.migrate_all_collections()

# migrate all articles

In [ ]:
## delete existing articles
# migration.delete_all_intercom_articles(are_you_sure=False)


In [ ]:
#res_articles = migration.migrate_all_articles(display_progress=True)

In [3]:
posts = CircleAPI.api_get_posts(render=False)
print(f'number of circle posts: {len(posts)}')
df_posts_circle = pd.DataFrame([{'circle_title': p['name'].strip(), 'circle_url': p['url'].strip()} for p in posts])

articles = IntercomAPI.api_get_articles(render=False)
print(f'number of intercom articles: {len(articles['data'])}')
df_articles_intercom = pd.DataFrame([{'intercom_title': a['title'].strip(), 'intercom_id': a['id'], 'intercom_url': a['url'].strip()} for a in articles['data']])

number of circle posts: 126
number of intercom articles: 125


In [4]:
df_articles = pd.merge(left = df_posts_circle, right = df_articles_intercom, left_on = 'circle_title', right_on='intercom_title', how = 'left')

In [5]:
bool_missing_intercom_titles = df_articles['intercom_title'].isna()
print(f'missing intercom articles: {bool_missing_intercom_titles.sum()}')
df_articles[bool_missing_intercom_titles]

missing intercom articles: 1


,circle_title,circle_url,intercom_title,intercom_id,intercom_url
12,Comment créer et paramétrer un projet ?,https://academiedidask.circle.so/c/creation/cr...,NaN,NaN,NaN


In [6]:
## replace the missing value
df_articles["intercom_url"].fillna("ARTICLE NON CREE SUR INTERCOM (Problème lors de l'importation)", inplace=True)

In [15]:
## change intercom URL to editing URL:
url_template = "https://app.intercom.com/a/apps/xqkc3jp0/articles/articles/{article_id}/show"
df_articles['intercom_url'] = df_articles['intercom_id'].apply(lambda article_id: url_template.format(article_id=article_id))


## save as google sheets

In [19]:
from gdrive.gdrive_api import GDrive
from datetime import datetime
gdrive = GDrive()
## save as excel file
df_final = df_articles[['circle_title', 'circle_url', 'intercom_url', 'intercom_id']].rename(columns={'circle_title': 'article_title'})

gsheet_title = f'migration-articles-circle-vers-intercom'
gsheet_id = gdrive.create_gsheet(gsheet_title)
gdrive.dump_df_in_gsheet(df_final, gsheet_id, 0)

In [ ]:
search_for = '31 août 2023'
b = df_articles_intercom['intercom_title'].str.contains(search_for)
df_articles_intercom[b]

In [ ]:
df_articles.iloc[21]['circle_title']

In [ ]:
df_articles_intercom.loc[b, 'intercom_title'].values

In [ ]:
df_articles_intercom

## migrate posts

## Posts

## Intercom

In [ ]:
intercom_collections = IntercomAPI.api_get_collections()

In [ ]:
res = IntercomAPI.api_create_collection(name="new collection")

In [ ]:
res = IntercomAPI.api_create_collection(name="aa", parent_id="8401934")

# SANDBOX

## Lookup table of articles - Circle vs Intercom

In [ ]:
is_problem_article = (df_articles['circle_title'] != df_articles['intercom_title'])
print(is_problem_article.sum())
df_articles[is_problem_article]

In [ ]:
df_articles['intercom_title'].duplicated().sum()

In [ ]:
[a['title'] for a in articles['data'] if "projet"  in a['title']]